In [13]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [18]:
!pip uninstall tensorflow protobuf tensorflow-estimator

Uninstalling tensorflow-1.14.0:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.14.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.14.0
Uninstalling protobuf-3.11.3:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/google/protobuf/*
    /usr/local/lib/python3.6/dist-packages/protobuf-3.11.3-py3.6-nspkg.pth
    /usr/local/lib/python3.6/dist-packages/protobuf-3.11.3.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled protobuf-3.11.3
Uninstalling tensorflow-estimator-1.14.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/tensorflow_estimator-1.14.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/*
Proceed (y/n)? y

In [19]:
!pip install tensorflow==1.14

  Using cached https://files.pythonhosted.org/packages/de/f0/96fb2e0412ae9692dbf400e5b04432885f677ad6241c088ccc5fe7724d69/tensorflow-1.14.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/57/02/5432412c162989260fab61fa65e0a490c1872739eb91a659896e4d554b26/protobuf-3.11.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl


# **LOAD DATA**

In [39]:
# Set paths of train and validation data
train_article_path = "train.article.txt"
train_title_path   = "train.title.txt"
 
import re
import string


def getSentList(path, n_sents):
  sentList=[]
  with open(path) as f:
    # using 100 to test the code only
    for line in f.readlines()[:n_sents]:
    #for line in f.readlines():
      # Remove begining and ending space
      sent=line.strip()
      # Replace ## or ##.#, #,## .. by #
      sent=re.sub("#(\W)*","num ",sent) 
      # Remove !#"$%&'()*+,-./:;<=>?@[\]^_`{|}~
      sent=sent.translate(str.maketrans(' ', ' ', string.punctuation))
      # Add sent to the list
      sentList.append(sent)
  
  return sentList

# Get train_article and train_title
train_article=getSentList(train_article_path,1000)
train_title=getSentList(train_title_path,1000)

# Print some data
print('Length of train data: ',len(train_article),len(train_title))

print('\nThe first five train articles and titles: ')
for i in range(5):
  print('Train article: ',train_article[i])
  print('Train title: ',train_title[i])
  print()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Length of train data:  1000 1000

The first five train articles and titles: 
Train article:  australia s current account deficit shrunk by a record num billion dollars lrb num billion us rrb in the june quarter due to soaring commodity prices  figures released monday showed 
Train title:  australian current account deficit narrows sharply

Train article:  at least two people were killed in a suspected bomb attack on a passenger bus in the strifetorn southern philippines on monday  the military said 
Train title:  at least two dead in southern philippines blast

Train article:  australian shares closed down num percent monday following a weak lead from the united states and lower commodity prices  dealers said 
Train title:  australian stocks close down num per

# **BUILD VOCAB**

In [42]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize

words=[]
# Need to modify this later
for sent in train_article+train_title:
  for word in word_tokenize(sent):
    if not word in stopwords.words():
      words.append(word)

# Print first 15 words
print("\nThe first 15 words: \n",words[:15])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

The first 15 words: 
 ['australia', 'current', 'account', 'deficit', 'shrunk', 'record', 'billion', 'dollars', 'lrb', 'billion', 'us', 'rrb', 'june', 'quarter', 'due']


In [43]:
import collections
import pickle
from pickle import dump
# Create word2int and int2word dictionary
word_counter = collections.Counter(words).most_common()

word2int = dict()
word2int["<pad>"] = 0
word2int["<unk>"] = 1
word2int["<s>"] = 2
word2int["</s>"] = 3
for word, _ in word_counter:
    word2int[word] = len(word2int)
    
int2word = dict(zip(word2int.values(), word2int.keys()))

print("The length of word2index: " , len(word2int))
print("\nThe word2index: \n" , word2int)
print("\nThe int2word: \n" , int2word)

print("\nsize:", len(int2word))

 
# Save word2int and int2word into pickle file

dump(word2int, open('vocab_w2i_1000.pkl', 'wb'))
dump(int2word, open('vocab_i2w_1000.pkl', 'wb'))

The length of word2index:  4563

The word2index: 
 {'<pad>': 0, '<unk>': 1, '<s>': 2, '</s>': 3, 'said': 4, 'thursday': 5, 'friday': 6, 'us': 7, 'percent': 8, 'president': 9, 'sunday': 10, 'new': 11, 'wednesday': 12, 'prices': 13, 'shares': 14, 'oil': 15, 'minister': 16, 'unk': 17, 'ike': 18, 'amid': 19, 'dealers': 20, 'two': 21, 'hurricane': 22, 'talks': 23, 'deal': 24, 'russia': 25, 'dollars': 26, 'bank': 27, 'party': 28, 'close': 29, 'european': 30, 'country': 31, 'police': 32, 'government': 33, 'russian': 34, 'people': 35, 'leader': 36, 'georgia': 37, 'white': 38, 'house': 39, 'obama': 40, 'south': 41, 'lehman': 42, 'power': 43, 'state': 44, 'share': 45, 'officials': 46, 'china': 47, 'zimbabwe': 48, 'attacks': 49, 'prime': 50, 'french': 51, 'gulf': 52, 'killed': 53, 'lower': 54, 'court': 55, 'chief': 56, 'month': 57, 'world': 58, 'closed': 59, 'week': 60, 'political': 61, 'would': 62, 'monday': 63, 'opposition': 64, 'visit': 65, 'official': 66, 'rivals': 67, 'economic': 68, 'texas'

# **CONVERT SEQ TO INT**

In [44]:
import numpy as np
from tensorflow import keras

# Convert input data from text to int
def get_intSeq(data_list,max_length,padding=False):
  seq_list=list()
  for sent in data_list:
    # Get tokens in each sent
    words=word_tokenize(sent)    
    
    # Use this for train_article
    if(padding):
      # Make all sent to have the same length as max_length
      if(len(words)<max_length):
        words=words + (max_length-len(words))*["<pad>"] 
      else:
        words=words[:max_length]
        
    # Use this for train_title
    else:
      words=words[:(max_length-1)]
    
    # Convert word to its corresponding int value
    # If the word doesnt exist, use the value of "<unk>" by default
    int_seq=[word2int.get(word,word2int["<unk>"]) for word in words]
    
    # Add int_seq to seq_list
    seq_list.append(int_seq)      
  
  return seq_list

# Define the max length of article and title
article_max_len = 30
title_max_len = 15

# Get the sequence of int value
train_article_intSeq=get_intSeq(train_article,article_max_len,padding=True)
train_title_intSeq=get_intSeq(train_title,title_max_len)

print('Train article: ',train_article[0])
print('Train article(int seq): ',train_article_intSeq[0])
print('\nTrain title: ',train_title[0])
print('Train title(int seq): ',train_title_intSeq[0])


Train article:  australia s current account deficit shrunk by a record num billion dollars lrb num billion us rrb in the june quarter due to soaring commodity prices  figures released monday showed 
Train article(int seq):  [615, 1, 961, 1893, 403, 2856, 1, 1, 962, 1, 142, 26, 207, 1, 142, 7, 208, 1, 1, 616, 812, 238, 1, 2857, 963, 13, 1170, 701, 63, 369]

Train title:  australian current account deficit narrows sharply
Train title(int seq):  [111, 961, 1893, 403, 4261, 490]


# **WORD2VEC**

In [56]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

# define training data
lines = []
for row in train_article+train_title: 
    lines.append([word for word in row.split()])

print(lines)
# train model
model = Word2Vec(lines, size=50, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
#print(model['the'])
# save model
model.save('model_w2v.bin')


[['australia', 's', 'current', 'account', 'deficit', 'shrunk', 'by', 'a', 'record', 'num', 'billion', 'dollars', 'lrb', 'num', 'billion', 'us', 'rrb', 'in', 'the', 'june', 'quarter', 'due', 'to', 'soaring', 'commodity', 'prices', 'figures', 'released', 'monday', 'showed'], ['at', 'least', 'two', 'people', 'were', 'killed', 'in', 'a', 'suspected', 'bomb', 'attack', 'on', 'a', 'passenger', 'bus', 'in', 'the', 'strifetorn', 'southern', 'philippines', 'on', 'monday', 'the', 'military', 'said'], ['australian', 'shares', 'closed', 'down', 'num', 'percent', 'monday', 'following', 'a', 'weak', 'lead', 'from', 'the', 'united', 'states', 'and', 'lower', 'commodity', 'prices', 'dealers', 'said'], ['south', 'korea', 's', 'nuclear', 'envoy', 'kim', 'sook', 'urged', 'north', 'korea', 'monday', 'to', 'restart', 'work', 'to', 'disable', 'its', 'nuclear', 'plants', 'and', 'stop', 'its', 'typical', 'brinkmanship', 'in', 'negotiations'], ['south', 'korea', 'on', 'monday', 'announced', 'sweeping', 'tax', 

Word2Vec(vocab=4722, size=50, alpha=0.025)
['australia', 's', 'current', 'account', 'deficit', 'shrunk', 'by', 'a', 'record', 'num', 'billion', 'dollars', 'lrb', 'us', 'rrb', 'in', 'the', 'june', 'quarter', 'due', 'to', 'soaring', 'commodity', 'prices', 'figures', 'released', 'monday', 'showed', 'at', 'least', 'two', 'people', 'were', 'killed', 'suspected', 'bomb', 'attack', 'on', 'passenger', 'bus', 'strifetorn', 'southern', 'philippines', 'military', 'said', 'australian', 'shares', 'closed', 'down', 'percent', 'following', 'weak', 'lead', 'from', 'united', 'states', 'and', 'lower', 'dealers', 'south', 'korea', 'nuclear', 'envoy', 'kim', 'sook', 'urged', 'north', 'restart', 'work', 'disable', 'its', 'plants', 'stop', 'typical', 'brinkmanship', 'negotiations', 'announced', 'sweeping', 'tax', 'reforms', 'including', 'income', 'corporate', 'cuts', 'boost', 'growth', 'stimulating', 'sluggish', 'private', 'consumption', 'business', 'investment', 'taiwan', 'share', 'wall', 'street', 'weakne

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [57]:
import numpy as np

# load model
word_embed_w2v = Word2Vec.load('model_w2v.bin')

# Sort the int2word
int2word_sorted=sorted(int2word.items())
 
# Get the list of word embedding corresponding to int value in ascending order
word_emb_list=list()
embedding_size=len(word_embed_w2v['the'])
for int_val, word in int2word_sorted:
  # Add Glove embedding if it exists
  if(word in word_embed_w2v):
    word_emb_list.append(word_embed_w2v[word])
  
  # Otherwise, the value of word embedding is 0
  else:
    word_emb_list.append(np.zeros([embedding_size], dtype=np.float32))
    
# Assign random vector to <s>, </s> token
word_emb_list[2] = np.random.normal(0, 1, embedding_size)
word_emb_list[3] = np.random.normal(0, 1, embedding_size)
 
# the final word embedding
word_embed=np.array(word_emb_list)
 
print("The first 5 word embedding: ")
for i in range(5):
  print(word_embed[i])

The first 5 word embedding: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[-1.40831492 -1.4097146  -1.47194795 -0.65404302 -0.62384367 -0.40793573
 -0.58201025  0.64025733  0.89410709  0.74314201  1.30828273  0.65208775
 -0.71655193  0.13247094 -1.09387791 -1.79489066 -1.70093097  0.28356127
 -1.06743899  1.20262624 -0.97449513 -0.43341734  0.43117497  0.96079203
 -0.45277063  0.46127549  0.90596621  0.94619312  1.38341845 -0.73966085
 -1.21843535 -0.96757037  0.59445948  1.09742103 -0.26920256 -0.36059596
 -1.18753694  0.50773747 -0.45339568 -0.03373569  1.38610139  1.84805844
  0.27771862 -1.28439935 -0.84975828  1.07564836  0.36287271  1.6234713
 -0.69875924 -0.01138478]
[-8.86367415e-01 -8.49994833e-01 -6.16137385e-01  1.67

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


# **GLOVE**

In [54]:
glove_path   = "glove.6B.50d.txt"
 
# Get the word embedding from Glove
word_emb_glove=dict()
with open(glove_path,encoding="utf8") as f:
  for line in f:
    el=line.split()
    word=el[0]
    emb=[float(val) for val in el[1:]]
    word_emb_glove[word]=emb

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
import numpy as np
# Sort the int2word
int2word_sorted=sorted(int2word.items())
 
# Get the list of word embedding corresponding to int value in ascending order
word_emb_list=list()
embedding_size=len(word_emb_glove['the'])
for int_val, word in int2word_sorted:
  # Add Glove embedding if it exists
  if(word in word_emb_glove):
    word_emb_list.append(word_emb_glove[word])
  
  # Otherwise, the value of word embedding is 0
  else:
    word_emb_list.append(np.zeros([embedding_size], dtype=np.float32))
    
# Assign random vector to <s>, </s> token
word_emb_list[2] = np.random.normal(0, 1, embedding_size)
word_emb_list[3] = np.random.normal(0, 1, embedding_size)
 
# the final word embedding
word_emb=np.array(word_emb_list)
 
print("The first 5 word embedding: ")
for i in range(5):
  print(word_emb[i])

The first 5 word embedding: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[ 8.45306020e-01  7.55365867e-01 -3.43773700e-01 -4.92453362e-01
 -5.38560387e-01 -3.92215794e-01  1.10052452e+00 -2.81680689e-01
 -1.42438930e+00  6.08097524e-01  5.59126705e-01 -2.70200588e-01
 -6.88430446e-01 -1.40108633e+00 -2.96710270e-01  3.12363385e-01
  2.66588653e-01 -5.16322157e-01 -1.00717400e+00 -1.45816703e-03
  6.54999460e-01  3.89564710e-01 -3.74703874e-01  7.79951530e-01
  2.39897731e-01  1.42135956e+00  1.37739480e+00  8.88334298e-01
 -3.93613886e+00 -1.74306886e+00  9.69810634e-01  1.14769786e+00
  4.04563782e-01 -1.96376489e+00  5.62748830e-01 -1.48460539e-01
  1.55625071e-01  6.88299372e-01  1.61651464e-01 -2.19671253e-01
  1.90738408e

In [0]:
import math
def get_batches(input_data, output_data, batch_size):
    #Convert input and output data from list to numpy array
    input_data=np.array(input_data)
    output_data=np.array(output_data)

    # Number of batches per epoch 
    num_batches_epoch = math.ceil(len(input_data)/batch_size)
    for batch_num in range(num_batches_epoch):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, len(input_data))
        yield input_data[start_index:end_index], output_data[start_index:end_index]

# **Model Seq2eq**

In [48]:
import tensorflow as tf
from tensorflow.contrib import rnn

class Seq2SeqModel(object):
    def __init__(self,vocab_size, word_embedding, input_len, output_len, params, train=True):
        # Get the vocab size
        self.vocab_size=vocab_size

        # Get hyper-parameters from params       
        self.num_layers=params['num_layers']
        self.num_hiddens=params['num_hiddens']    
        self.learning_rate = params['learning_rate']
        self.keep_prob = params['keep_prob']
        self.beam_width = params['beam_width']

        # Using BasicLSTMCell as a cell unit
        self.cell=tf.nn.rnn_cell.LSTMCell

        # Define Place holders for the model
        self.batch_size=tf.placeholder(tf.int32,(),name="batch_size")
        self.global_step = tf.Variable(0, trainable=False) # False means not adding the variable to the graph collection 

        # place holders for encoder
        self.inputSeq=tf.placeholder(tf.int32,[None,input_len])
        self.inputSeq_len=tf.placeholder(tf.int32, [None]) # Need to define the Shape as required in tf.contrib.seq2seq.tile_batch

        # place holders for decoder
        self.decoder_input=tf.placeholder(tf.int32,[None,output_len])
        self.decoder_len=tf.placeholder(tf.int32, [None])
        self.decoder_target=tf.placeholder(tf.int32,[None,output_len])

        # Define projection_layer
        self.projection_layer = tf.layers.Dense(self.vocab_size, use_bias=False)

        # Define the Embedding layer
        with tf.name_scope("embedding"):
            self.embeddings=tf.get_variable("embeddings",initializer=tf.constant(word_embedding,dtype=tf.float32))

            # map the int value with its embeddings
            input_emb=tf.nn.embedding_lookup(self.embeddings,self.inputSeq)
            decoder_input_emb=tf.nn.embedding_lookup(self.embeddings,self.decoder_input)

            # Convert from batch_size*seq_len*embedding to seq_len*batch_size*embedding to feed data with timestep      
            # But, we need to set time_major=True during Training
            self.encoder_inputEmb = tf.transpose(input_emb, perm=[1, 0, 2])
            self.decoder_inputEmb = tf.transpose(decoder_input_emb, perm=[1, 0, 2])

        # Define the Encoder
        with tf.name_scope("encoder"):      
            # Create RNN Cell for forward and backward direction
            fw_cells=list()
            bw_cells=list()
            for i in range(self.num_layers):
                fw_cell= self.cell(self.num_hiddens)
                bw_cell= self.cell(self.num_hiddens)

                # Add Dropout
                fw_cell=rnn.DropoutWrapper(fw_cell,output_keep_prob=self.keep_prob)
                bw_cell=rnn.DropoutWrapper(bw_cell,output_keep_prob=self.keep_prob)

                # Add cell to the list
                fw_cells.append(fw_cell)
                bw_cells.append(bw_cell)


            # Build a multi bi-directional model from fw_cells and bw_cells
            outputs, encoder_state_fw, encoder_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
              cells_fw=fw_cells, cells_bw=bw_cells,inputs=self.encoder_inputEmb,time_major=True, sequence_length=self.inputSeq_len, dtype=tf.float32)

            # The ouput of Encoder (time major)
            self.encoder_outputs=outputs

            # Use the final state of the last layer as encoder_final_state 
            encoder_state_c = tf.concat((encoder_state_fw[-1].c, encoder_state_bw[-1].c), 1)
            encoder_state_h = tf.concat((encoder_state_fw[-1].h, encoder_state_bw[-1].h), 1)
            self.encoder_final_state = rnn.LSTMStateTuple(c=encoder_state_c, h=encoder_state_h)
      
        # Define the Decoder for training
        with tf.name_scope("decoder"):
            # Define Decoder cell
            decoder_num_hiddens =self.num_hiddens * 2 # As we use bi-directional RNN
            decoder_cell=self.cell(decoder_num_hiddens)
      
            # Training mode 
            if(train):
                # Convert from time major to batch major 
                attention_states = tf.transpose(self.encoder_outputs, [1, 0, 2])

                # Decoder with attention      
                attention=tf.contrib.seq2seq.BahdanauAttention(num_units=decoder_num_hiddens, memory=attention_states, memory_sequence_length=self.inputSeq_len,normalize=True)
                attention_decoder_cell= tf.contrib.seq2seq.AttentionWrapper(cell=decoder_cell,attention_mechanism=attention,attention_layer_size=decoder_num_hiddens)

                # Use the final state of encoder as the initial state of the decoder
                decoder_initial_state = attention_decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size)
                decoder_initial_state = decoder_initial_state.clone(cell_state=self.encoder_final_state )

                # Use TrainingHelper to train the Model 
                training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=self.decoder_inputEmb,sequence_length=self.decoder_len, time_major=True)
                decoder = tf.contrib.seq2seq.BasicDecoder(cell=attention_decoder_cell,helper=training_helper,initial_state=decoder_initial_state,output_layer=self.projection_layer)
                logits, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=True,maximum_iterations=output_len)


                # Convert from time major to batch major 
                self.training_logits = tf.transpose(logits.rnn_output, perm=[1, 0, 2])

                # Adding zero to make sure training_logits has shape: [batch_size, sequence_length, num_decoder_symbols]
                self.training_logits = tf.concat([self.training_logits, tf.zeros([self.batch_size, output_len - tf.shape(self.training_logits)[1], self.vocab_size])], axis=1)

            # Inference mode 
            else:
                # Using Beam search
                tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(tf.transpose(self.encoder_outputs, perm=[1, 0, 2]), multiplier=self.beam_width)
                tiled_encoder_final_state=tf.contrib.seq2seq.tile_batch(self.encoder_final_state, multiplier=self.beam_width)
                tiled_inputSeq_len=tf.contrib.seq2seq.tile_batch(self.inputSeq_len, multiplier=self.beam_width)

                # Decoder with attention with Beam search
                attention=tf.contrib.seq2seq.BahdanauAttention(num_units=decoder_num_hiddens, memory=tiled_encoder_outputs, memory_sequence_length=tiled_inputSeq_len,normalize=True)
                attention_decoder_cell= tf.contrib.seq2seq.AttentionWrapper(cell=decoder_cell,attention_mechanism=attention,attention_layer_size=decoder_num_hiddens)

                # Use the final state of encoder as the initial state of the decoder
                decoder_initial_state = attention_decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size * self.beam_width)
                decoder_initial_state = decoder_initial_state.clone(cell_state=tiled_encoder_final_state)

                # Build a Decoder with Beam Search
                beamSearch_decoder=tf.contrib.seq2seq.BeamSearchDecoder(          
                    cell=attention_decoder_cell,
                    embedding=self.embeddings,
                    start_tokens=tf.fill([self.batch_size],tf.constant(2)),
                    end_token=tf.constant(3),
                    initial_state=decoder_initial_state,
                    beam_width=self.beam_width,
                    output_layer=self.projection_layer  
                )

                # Perform dynamic decoding with beamSearch_decoder
                outputs, _ , _ =tf.contrib.seq2seq.dynamic_decode(decoder=beamSearch_decoder,maximum_iterations= output_len,output_time_major=True)

                # Convert from seq_len*batch_size*beam_width to batch_size*beam_width*seq_len
                outputs=tf.transpose(outputs.predicted_ids, perm=[1, 2, 0])

                # Take the first beam (best result) as Decoder ouput 
                self.decoder_outputs=outputs[:,0,:]

        with tf.name_scope("optimization"):
            # Used for Training mode only 
            if(train):
                # Caculate loss value 
                masks = tf.sequence_mask(lengths=self.decoder_len,maxlen=output_len, dtype=tf.float32)         
                self.loss = tf.contrib.seq2seq.sequence_loss(logits=self.training_logits,targets=self.decoder_target,weights=masks)

                # Using AdamOptimizer
                optimizer = tf.train.AdamOptimizer(self.learning_rate)
                # Compute gradient 
                gradients = optimizer.compute_gradients(self.loss)
                # Apply Gradient Clipping 
                gradients_clipping = [(tf.clip_by_value(grad, clip_value_min=-5., clip_value_max=5.), var) for grad, var in gradients if grad is not None]

                # Apply gradients to variables
                self.train_update = optimizer.apply_gradients(gradients_clipping, global_step=self.global_step)
        
print("done")

done


# **TRAIN MODEL**

In [58]:
import tensorflow as tf
# Run the Seq2seq model written in another ipynb file 

# Define hyper-parameters for the Model
params=dict()
params['num_layers']=2
params['num_hiddens']=400
params['learning_rate']=0.001
params['keep_prob']=0.85
params['beam_width']=5

num_epochs=10
early_stop=5 # Stop if there is no improvement after 5 epochese

# Set paths to save the model
checkpoint = "TSModel.ckpt"
import time
start_time=time.time()


tf.reset_default_graph()
with tf.Session() as sess:
    # Create a Seq2seq model
    model=Seq2SeqModel(len(int2word), word_embed, article_max_len, title_max_len, params)
  
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    min_loss=1000 # To find the minimum loss during training   
    no_impove_count=0 # Count the number of consecutive epoch having no improvement
    
    for epoch in range (num_epochs):  
        # Get batches from training data 
        batches=get_batches(train_article_intSeq,train_title_intSeq,batch_size=50) 
    
        # Reset epoch_loss after each epoch 
        epoch_loss=0
        # Interate over batches
        for batch_i,(batch_x, batch_y) in enumerate(batches):
            # The actual length of each sequence in the batch (excluding "<pad>")
            batch_x_len = list(map(lambda seq: len([word_int for word_int in seq if word_int != 0]), batch_x))

            # Decoder input is created by adding <s> to the begining of each output sentence 
            batch_decoder_input = list(map(lambda seq: [word2int["<s>"]] + list(seq), batch_y))

            # The actual length of each Decoder input (excluding "<pad>")
            batch_decoder_len = list(map(lambda seq: len([word_int for word_int in seq if word_int != 0]), batch_decoder_input))

            # The actual ouput of Decoder is created by adding </s> to the begining of each output sentence
            batch_decoder_output = list(map(lambda seq: list(seq) + [word2int["</s>"]], batch_y))

            # Add <pad> to make all input and ouput of Decoder have same length
            batch_decoder_input = list(
                map(lambda seq: seq + (title_max_len - len(seq)) * [word2int["<pad>"]], batch_decoder_input))
            batch_decoder_output = list(
                map(lambda seq: seq + (title_max_len - len(seq)) * [word2int["<pad>"]], batch_decoder_output))

            # Create a train_feed_dict
            train_feed_dict = {
                model.batch_size: len(batch_x),
                model.inputSeq: batch_x,
                model.inputSeq_len: batch_x_len,

                model.decoder_input: batch_decoder_input,
                model.decoder_len: batch_decoder_len,
                model.decoder_target: batch_decoder_output
            }
      
            # Start training the model
            _, step, loss,encoder_outputs = sess.run([model.train_update, model.global_step, model.loss,model.encoder_outputs], feed_dict=train_feed_dict)
            epoch_loss+=loss
      
            # Display loss value of each step
            print("step {0}: loss = {1}".format(step, loss))      
   
        print("Finish epoch",epoch+1 )
        # Averaging the epoch_loss
        epoch_loss=epoch_loss/(batch_i+1)
    
        # Save the model if the epoch_loss is at a new minimum,
        if epoch_loss <= min_loss:
            # Set new minimum loss
            min_loss=epoch_loss
            # Reset the no_impove_count
            no_impove_count=0 
      
            # Save the new model
            saver = tf.train.Saver(tf.global_variables()) 
            saver.save(sess, checkpoint)
      
            print('New model saved, minimum loss:',min_loss,'\n') 
      
        # Early stopping
        else:
            print("No Improvement!",'\n')
            no_impove_count+=1
            if(no_impove_count==early_stop):
                print("Early stopping... Finish training")
                break

end_time=time.time()
training_time=(end_time-start_time)/60
print("\nTraining time (mins): ",training_time )

step 1: loss = 8.42259407043457
step 2: loss = 8.200321197509766
step 3: loss = 6.658049583435059
step 4: loss = 7.158230304718018
step 5: loss = 7.5550031661987305
step 6: loss = 6.959869861602783
step 7: loss = 6.8960771560668945
step 8: loss = 6.532432556152344
step 9: loss = 6.078763008117676
step 10: loss = 6.497953414916992
step 11: loss = 6.903436183929443
step 12: loss = 6.381427764892578
step 13: loss = 6.223559379577637
step 14: loss = 6.238027095794678
step 15: loss = 6.366315841674805
step 16: loss = 5.847128391265869
step 17: loss = 6.121161937713623
step 18: loss = 6.402615070343018
step 19: loss = 6.327394485473633
step 20: loss = 6.182215213775635
Finish epoch 1
New model saved, minimum loss: 6.697628784179687 

step 21: loss = 5.718019962310791
step 22: loss = 5.920468807220459
step 23: loss = 5.378968715667725
step 24: loss = 5.73895788192749
step 25: loss = 5.57590389251709
step 26: loss = 5.709475517272949
step 27: loss = 5.691187858581543
step 28: loss = 5.33215665

In [59]:
valid_article_path = "valid.article.filter.txt"
valid_title_path   = "valid.title.filter.txt"

# The hyper-parameters of the Model (should be the same as in Training)
params=dict()
params['num_layers']=2
params['num_hiddens']=400
params['learning_rate']=0.001
params['keep_prob']=0.85
params['beam_width']=5
 
# Path of the saved model
checkpoint = "TSModel.ckpt"
 
# Reset the default graph
tf.reset_default_graph()
 
# Get the first 10 validation articles and validation titles
valid_article=getSentList(valid_article_path,10)
valid_title=getSentList(valid_title_path,10)
 
# Get the sequence of int value
valid_article_intSeq=get_intSeq(valid_article,article_max_len,padding=True)
valid_title_intSeq=get_intSeq(valid_title,title_max_len)
 
with tf.Session() as sess:  
  # Load saved model 
  # Use Seq2SeqModel to create the same graph as saved model
  loaded_model=Seq2SeqModel(len(int2word), word_embed, article_max_len, title_max_len, params, train=False)
  
  # Load the value of variables in saved model
  saver = tf.train.Saver(tf.global_variables())
  saver.restore(sess, checkpoint)
  
  # Get batches from validation data 
  batches_valid=get_batches(valid_article_intSeq,valid_title_intSeq,batch_size=50) 
  
  # Interate over batches
  for batch_valid_i,(batch_x_valid, batch_y_valid) in enumerate(batches_valid):
    # The actual length of each sequence in the batch (excluding "<pad>")
    batch_x_len_valid = list(map(lambda seq: len([word_int for word_int in seq if word_int != 0]), batch_x_valid))
    
    # Create a feed_dict for validation data
    valid_feed_dict = {
            loaded_model.batch_size: len(batch_x_valid),
            loaded_model.inputSeq: batch_x_valid,
            loaded_model.inputSeq_len: batch_x_len_valid,
        }
    
    # Get the decoder output by Inference
    decoder_outputs=sess.run(loaded_model.decoder_outputs,feed_dict=valid_feed_dict)
    
    # Convert from sequence of int to actual sentence
    output_titles=[]    
    # Loop through each seq in decoder_outputs
    for out_seq in decoder_outputs:  
      out_sent=list()
      for word_int in out_seq:    
        # Convert int to word 
        word=int2word[word_int]  
        # Stop converting when it reach to the end of ouput sentence
        if word == "</s>":
          break
        else:
          out_sent.append(word)
      # Combine list of word to sentence and add this sentence to output_titles
      output_titles.append(" ".join(out_sent))
      
# Display the results      
for i in range(10):
  print("Article: ",valid_article[i])
  print("Actual title: ",valid_title[i])
  print("Generated title: ",output_titles[i],'\n')

INFO:tensorflow:Restoring parameters from TSModel.ckpt


INFO:tensorflow:Restoring parameters from TSModel.ckpt


Article:  fivetime world champion michelle kwan withdrew from the num us figure skating championships on wednesday  but will petition us skating officials for the chance to compete at the num turin olympics 
Actual title:  injury leaves kwan s olympic hopes in limbo
Generated title:  <unk> <unk> <unk> <unk> 

Article:  us business leaders lashed out wednesday at legislation that would penalize companies for employing illegal immigrants 
Actual title:  us business attacks tough immigration law
Generated title:  search <unk> <unk> <unk> <unk> <unk> 

Article:  general motors corp said wednesday its us sales fell num percent in december and four percent in num with the biggest losses coming from passenger car sales 
Actual title:  gm december sales fall num percent
Generated title:  china <unk> <unk> <unk> <unk> <unk> 

Article:  several thousand people gathered on wednesday evening on the main square in zagreb for a public draw and an open air party to celebrate the croatian capital s se